# Branch and Bound algorithm for general system function

A system function needs to be coherent, i.e. (1) a high state of component events mean their better performance, and (2) a better (worse) component vector state does not lead to a worse (better) system state.

## Example problem

For example, consider a transport system 

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import copy

from BNS_JT import trans, variable, branch

HOME = os.getcwd()

# Network
node_coords = {'n1': (-2, 3),
               'n2': (-2, -3),
               'n3': (2, -2),
               'n4': (1, 1),
               'n5': (0, 0)}

arcs = {'e1': ['n1', 'n2'],
	'e2': ['n1', 'n5'],
	'e3': ['n2', 'n5'],
	'e4': ['n3', 'n4'],
	'e5': ['n3', 'n5'],
	'e6': ['n4', 'n5']}

arcs_avg_kmh = {'e1': 40,
                'e2': 40,
                'e3': 40,
                'e4': 30,
                'e5': 30,
                'e6': 20}

arc_lens_km = trans.get_arcs_length(arcs, node_coords)
arc_times_h = {k: v/arcs_avg_kmh[k] for k, v in arc_lens_km.items()}

comps_name = [k for k in arcs] # *TODO* this is not necessary if branch's down and up are defined by dictionary (instead of list)


od_pair = ('n1', 'n3')


# Component events
no_arc_st = 3 # number of component states 
delay_rat = [10, 2, 1] # delay in travel time given each component state (ratio)
vari = {}
for k, v in arcs.items():
    vari[k] = variable.Variable( name=k, B = np.eye( no_arc_st ), values = [arc_times_h[k]*np.float64(x) for x in delay_rat])


# plot graph
G = nx.Graph()
for k, x in arcs.items():
    G.add_edge(x[0], x[1], time=arc_times_h[k], label=k)

for k, v in node_coords.items():
    G.add_node(k, pos=v)

pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label')

fig = plt.figure()
ax = fig.add_subplot()
nx.draw(G, pos, with_labels=True, ax=ax)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
fig.savefig(os.path.join(HOME, 'graph.png'), dpi=200)

In [3]:
# Get travel time between a given OD pair and the shortest path taken
def get_time_and_path( comps_st, od_pair, arcs, vari ):
    G = nx.Graph()
    for k, x in arcs.items():
        c_st = comps_st[k]
        G.add_edge(x[0], x[1], time=vari[k].values[c_st-1])

    path = nx.shortest_path( G, source = od_pair[0], target = od_pair[1], weight = 'time' )
    time = nx.shortest_path_length( G, source = od_pair[0], target = od_pair[1], weight = 'time' )

    return time, path

## System function

A system function needs to return (1) system function value, (2) system state, and (3) minimally required component state to fulfill the obtained system function value. If (3) is unavailable, it can be returned as None. <br>
It requires input being a component state in regard to which a system analysis needs to be done.

Two things to note about a system function: <br>
(1) It is preferred if a system function returns a possible scenario of components performance that has the highest probability of occurrence. <br>
(2) It is preferred if one can obtain a minimum required state of component vector from a result of system analysis. In case that this is not possible, the reference component vector for decomposition becomes the input state of component vector (which work okay by the proposed branch and bound algorithm).

An example system function is as follows. <br>
Here, We define the system failure event as a travel time longer than thres*(normal time). <br>
It is noted that for a failure event, it returns 'None' for minimal (failure) rule since there is no efficient way to identify one.

In [4]:
def sf_min_path( comps_st, od_pair, arcs, vari, thres, sys_val_itc ):

    sys_val, path = get_time_and_path( comps_st, od_pair, arcs, vari )

    if sys_val > thres*sys_val_itc:
        sys_st = 'fail'
    else:
        sys_st = 'surv'

    if sys_st == 'surv': # in this case we know how to find out minimally required component state
        min_comps_st = {}
        for i in range(len(path)-1):
            nodes_i = [path[i], path[i+1]]
            nodes_i_rev = [path[i+1], path[i]] # reversed pair (arcs are bi-directional)
            arc_i = next((k for k, v in arcs.items() if v == nodes_i or v == nodes_i_rev), None)
            min_comps_st[arc_i] = comps_st[arc_i]
    
    else: # sys_st == 'fail'
        min_comps_st = None

    return sys_val, sys_st, min_comps_st


In [5]:
# Intact state of component vector
comps_st_itc = {k: len(vari[k].B[0]) for k in arcs} # intact state (i.e. the highest state)
sys_val_itc, path_itc = get_time_and_path( comps_st_itc, od_pair, arcs, vari )

# defines the system failure event
thres = 2 

# Given a system function, i.e. sf_min_path, it should be represented by a function that only has "comps_st" as input.
sys_fun = lambda comps_st : sf_min_path( comps_st, od_pair, arcs, vari, thres, sys_val_itc ) # TODO: branch needs to have states defined in dictionary instead of list.


In [6]:
# Example 1
sys_val, sys_st, min_comps_st = sys_fun( {'e1': 3, 'e2': 1, 'e3': 2, 'e4': 3, 'e5': 3, 'e6': 3} )
print(sys_val, sys_st, min_comps_st)

# Example 2
sys_val, sys_st, min_comps_st = sys_fun(  {'e1': 1, 'e2': 2, 'e3': 1, 'e4': 1, 'e5': 1, 'e6': 3} )
print(sys_val, sys_st, min_comps_st)

# Example 3
sys_val, sys_st, min_comps_st = sys_fun(  {'e1': 3, 'e2': 3, 'e3': 1, 'e4': 2, 'e5': 2, 'e6': 3} )
print(sys_val, sys_st, min_comps_st)

0.4245584679314058 fail None
1.1230866053552628 fail None
0.2787005902030124 surv {'e2': 3, 'e5': 2}


## Branch and Bound algorithm

### Functions

In [7]:
def rule_dict_to_list( r_dict, r_st, no_comp, comps_name_list, worst_st = 1 ):
    
    if r_st == 'surv':
        r_list = [worst_st] * no_comp

        for i in range( no_comp ):
            name_i = comps_name_list[i]

            if name_i in r_dict:
                r_list[i] = r_dict[name_i]
    
    else: # r_st == 'fail'
    
        r_list = [best_st] * no_comp

        for i in range( no_comp ):
            name_i = comps_name_list[i]

            if name_i in r_dict:
                r_list[i] = r_dict[name_i]

    return r_list


In [8]:
# function that converts list representation of a component vector state to dictionary

def comps_st_list_to_dict( st_list, comps_name_list ):
    
    st_dict = {comps_name_list[i]: st_list[i] for i in range( len(st_list) )}

    return st_dict

def comps_st_dict_to_list( st_dict, comps_name_list ):

    no_comp = len( comps_name_list )
    st_list = [None] * no_comp
    for i in range(no_comp):
        name_i = comps_name_list[i]
        st_list[ i ] = st_dict[name_i]

    return st_list


In [9]:
def get_compat_rules( cst_dict, rules, rules_st ):

    #cst_dict: component vector state in dictionary
    #rules: list of rules in dictionary
    #rules_st: list of rules' state (either 'surv' or 'fail') -- must have the same length as rules

    cr_inds = [] # compatible rules--stored by their indices
    cst_state = 'unk' # unknown
    for ind, r in enumerate(rules):
        if rules_st[ind] == 'surv':
            if all( [ cst_dict[k] >= r[k] for k in r] ): # the survival rule is satisfied

                cr_inds.append( ind )
                cst_state = 'surv'

        else: # rules_st[ind] == 'fail'
            if all( [ cst_dict[k] <= r[k] for k in r] ): # the failure rule is compatible

                cr_inds.append( ind )
                cst_state = 'fail'
            
    return cr_inds, cst_state


In [10]:
# Example 1: at the beginning of branch (with no known rules)
rules = []
rules_st = []

up_dict = {k: len(vari[k].B[0]) for k in arcs}
cr_inds, cst_state = get_compat_rules( up_dict, rules, rules_st )
print(cr_inds, cst_state)

down_dict = {k: 1 for k in arcs}
cr_inds, cst_state = get_compat_rules( down_dict, rules, rules_st )
print(cr_inds, cst_state)

# Example 2
rules += [{'e2': 2, 'e5': 3}]
rules_st += ['surv']
cr_inds, cst_state = get_compat_rules( up_dict, rules, rules_st )
print(cr_inds, cst_state)

cr_inds, cst_state = get_compat_rules( down_dict, rules, rules_st )
print(cr_inds, cst_state)

# Exmple 3
rules += [{'e1':2, 'e2': 1}]
print(rules)
rules_st += ['fail']
print(rules_st)
cr_inds, cst_state = get_compat_rules( up_dict, rules, rules_st )
print(cr_inds, cst_state)

cr_inds, cst_state = get_compat_rules( down_dict, rules, rules_st )
print(cr_inds, cst_state)

# Example 4
cst = {'e1':1, 'e2': 1, 'e3': 3, 'e4': 2, 'e5': 1, 'e6': 3}
cr_inds, cst_state = get_compat_rules( cst, rules, rules_st )
print(cr_inds, cst_state)

cr_inds, cst_state = get_compat_rules( cst, rules, rules_st )
print(cr_inds, cst_state)


[] unk
[] unk
[0] surv
[] unk
[{'e2': 2, 'e5': 3}, {'e1': 2, 'e2': 1}]
['surv', 'fail']
[0] surv
[1] fail
[1] fail
[1] fail


In [11]:
# Update a rules list by removing dominated rules and adding a new rule
def add_a_new_rule( rules_list, rules_st, rule1, fail_or_surv ):

    r_rmv_inds = []
    add_rule1 = True
    for i in range(len(rules_list)):
        r_i = rules_list[i]

        if all( [True if k in r_i else False for k in rule1.keys()] ): # does all keys in rule 1 exist for rule_i?

            if fail_or_surv == 'surv' and rules_st[i] == 'surv':            
                if all( [True if r_i[k] >= v else False for k,v in rule1.items()] ): # this rule is dominated by the new rule
                    r_rmv_inds += [i]           
                elif all( [True if k in rule1 else False for k in r_i.keys()] ) and all( [True if rule1[k] >= v else False for k,v in r_i.items()] ):
                    add_rule1 = False
                    break # the new rule is dominated by an existing one. no further investigation required (assuming that a given list is a set of non-dominated rules)
                
            elif fail_or_surv == 'fail' and rules_st[i] == 'fail':              
                if all( [True if r_i[k] <= v else False for k,v in rule1.items()] ): # this rule is dominated by the new rule
                    r_rmv_inds += [i]
                elif all( [True if k in rule1 else False for k in r_i.keys()] ) and all( [True if rule1[k] <= v else False for k,v in r_i.items()] ):
                    add_rule1 = False
                    break # the new rule is dominated by an existing one. no further investigation required (assuming that a given list is a set of non-dominated rules)

    rules_list_new = copy.deepcopy( rules_list )
    rules_st_new = copy.deepcopy(rules_st)
    for i in r_rmv_inds[::-1]:
        try:
            del rules_list_new[i]
            del rules_st_new[i]
        except:
            ddd = 1

    if add_rule1 == True:
        rules_list_new += [rule1]
        rules_st_new += [fail_or_surv]

    return rules_list_new, rules_st_new



In [12]:
# Example 1: survival case
rules = [{'e2': 3, 'e5': 3}, {'e2': 2, 'e5': 2}] # a list of rules
rules_st = ['surv', 'fail']
rs_new = {'e2': 2, 'e5': 3} # a new rule
rs_new_st = 'surv'

rules, rules_st = add_a_new_rule( rules, rules_st, rs_new, rs_new_st )
print(rules, rules_st)

# Exmaple 2: failure case
rf_new = {'e2': 2, 'e5': 1} # a new rule
rf_new_st = 'fail'

rules, rules_st = add_a_new_rule( rules, rules_st, rf_new, rf_new_st )
print(rules, rules_st)

[{'e2': 2, 'e5': 2}, {'e2': 2, 'e5': 3}] ['fail', 'surv']
[{'e2': 2, 'e5': 2}, {'e2': 2, 'e5': 3}] ['fail', 'surv']


In [20]:
# get the next component and its state for branching
def get_comp_st_for_next_bnb( up_dict, down_dict, rules, rules_st ):

    # rules: a list of rules (in dictionary)
    # rules_st: a list of rules' state (the same length as rules)

    cr_inds_up, _ = get_compat_rules( up_dict, rules, rules_st ) 
    cr_inds_down, _ = get_compat_rules( down_dict, rules, rules_st ) 

    cr_inds = set( cr_inds_up + cr_inds_down )
    c_rules = [rules[i] for i in cr_inds]
    c_rules_st = [rules_st[i] for i in cr_inds]
    
    r_len = [len(x) for x in c_rules]
    r_len_sort_ind = [i[0] for i in sorted(enumerate(r_len), key=lambda x:x[1])]

    comps_cnt = {}
    comp_bnb = None
    for i in r_len_sort_ind:
        r_i = c_rules[i]
        r_i_st = c_rules_st[i]
        comps_i = [k for k in r_i]

        comps_i_cnt = [] # counts of components' appearance across rules
        for x in comps_i:
            if x not in comps_cnt:
                x_cnt = sum( [1 if x in r else 0 for r in c_rules] )
                comps_cnt[x] = x_cnt
            else:
                x_cnt = comps_cnt[x]

            comps_i_cnt.append(x_cnt)

        
        c_i_sort_ind = [j[0] for j in sorted(enumerate(comps_i_cnt), key=lambda x:x[1])] # order components by their frequency in rules set
        for j in c_i_sort_ind:
            x_ij = comps_i[j]
            x_ij_st = r_i[x_ij]

            if r_i_st == 'surv':
                if x_ij_st > down_dict[x_ij]:
                    comp_bnb = x_ij
                    st_bnb_up = x_ij_st # this is always the upper branch's lower state
                    break

            else: # r_i_st == 'fail'
                if x_ij_st < up_dict[x_ij]:
                    comp_bnb = x_ij
                    st_bnb_up = x_ij_st + 1 # this is always the upper branch's lower state
                    break

        if comp_bnb is not None:
            break

    # in case nothing has been selected from above
    if comp_bnb is None: # just randomly select from the components that have higher frequencies
        for r in rules:
            for k in r:
                if k not in comps_cnt:
                    comps_cnt[k] = sum( [1 if x in r else 0 for r in rules] )


        cnt_sort_xs = sorted( comps_cnt )
        for x in cnt_sort_xs:
            if down_dict[x] < up_dict[x]:
                comp_bnb = x
                st_bnb_up = up_dict[x]
                break


    return comp_bnb, st_bnb_up

In [14]:
# Example 1
rules = [{'e2': 3, 'e5': 3}, {'e1': 1, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 1, 'e6': 1}]
rules_st = ['surv', 'fail']

up_dict = {k: len(vari[k].B[0]) for k in arcs}
down_dict = {k:1 for k in arcs}

comp_bnb, st_bnb_up = get_comp_st_for_next_bnb( up_dict, down_dict, rules, rules_st )
print( comp_bnb, st_bnb_up )



e2 3


In [15]:
def decomp_to_two_branches( br1, comp_bnb, st_bnb_up, comps_name_list ):

    down_dict = comps_st_list_to_dict( br1.down, comps_name_list )
    up_dict = comps_st_list_to_dict( br1.up, comps_name_list )

    up_dict_bl = copy.deepcopy(up_dict) # the branch on the lower side
    up_dict_bl[comp_bnb] = st_bnb_up - 1

    down_dict_bu = copy.deepcopy(down_dict) # the branch on the upper side
    down_dict_bu[comp_bnb] = st_bnb_up
        
    up_list_bl = comps_st_dict_to_list( up_dict_bl, comps_name_list )
    down_list_bu = comps_st_dict_to_list(down_dict_bu, comps_name_list)
    new_brs_list = [branch.Branch( br1.down, up_list_bl, is_complete=False ),
                    branch.Branch( down_list_bu, br1.up, is_complete=False )]
    
    return new_brs_list



In [16]:
# Example 1
comp_bnb = 'e2'
st_bnb_up = 3

br1 = branch.Branch( [1] * len(arcs), [no_arc_st] * len(arcs), is_complete=False )
new_brs = decomp_to_two_branches( br1, comp_bnb, st_bnb_up, comps_name )
print(new_brs)


# Exmple 2
comp_bnb = 'e2'
st_bnb_up = 2

br1 = branch.Branch( [1] * len(arcs), [no_arc_st] * len(arcs), is_complete=False )
new_brs = decomp_to_two_branches( br1, comp_bnb, st_bnb_up, comps_name )
print(new_brs)

[Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 2, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None, Branch(down=[1, 3, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None]
[Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 1, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None, Branch(down=[1, 2, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None]


### Initialisation

In [32]:
max_br = 1000 # user input: maximum no. of branches 
no_sf = 0 # number of system function runs so far
sys_res = pd.DataFrame(data={'sys_val': [], 'comps_st': [], 'comps_st_min': []}) # system function results 

rules = [] # a list of known rules
rules_st = [] # a list of known rules' states

### Execution

In [33]:
no_iter =  0
brs = [branch.Branch( [], [], is_complete=False )] # dummy branch to start the while loop
while sum( [1 if b.is_complete == False else 0 for b in brs ] ) > 0: # for debugging 

    ### for debugging ###
    no_iter += 1 
    print( 'Iteration: ', no_iter ) 

    print( 'Known rules: ' )
    for r,s in zip(rules, rules_st):
        print( s, r )
    #####################

    ## Start from the total event ##
    down_list = [1] * len(arcs) # all components in the worst state
    up_list = [len( vari[e].B[0] ) for e in comps_name] # all components in the best state

    brs = [branch.Branch( down_list, up_list, is_complete=False )]


    up_dict = comps_st_list_to_dict( up_list, comps_name )
    cr_inds1, cst_state_up = get_compat_rules( up_dict, rules, rules_st )
    brs[0].up_state = cst_state_up

    down_dict = comps_st_list_to_dict( down_list, comps_name )
    cr_inds1, cst_state1_down = get_compat_rules( down_dict, rules, rules_st )
    brs[0].down_state = cst_state1_down
    ###############################


    no_br_stage = 0 # for debugging
    stop_br = False
    while sum( [1 if b.is_complete == False else 0 for b in brs ] ) > 0:

        ### for debugging ###
        no_br_stage += 1 
        print( 'Branching stage: ', no_br_stage ) 
        #####################
        
        brs_new = []

        for i in range(len( brs )):
            
            try:
                br_i = brs[i]
                up_dict = comps_st_list_to_dict( br_i.up, comps_name )
                down_dict = comps_st_list_to_dict( br_i.down, comps_name )

                cr_inds_up_i, up_st = get_compat_rules( up_dict, rules, rules_st )
                cr_inds_down_i, down_st = get_compat_rules( down_dict, rules, rules_st )

            except:
                ddd = 1

            ### for debugging ###
            print( 'Branch #: ', i+1 ) 
            print( 'Branch: ', br_i )
            #####################

            if br_i.up_state == 'unk' and len(cr_inds_up_i) == 0:
                cst_list = br_i.up # perform analysis on this state
                stop_br = True
                break 

            elif br_i.down_state == 'unk' and len(cr_inds_down_i) == 0:
                cst_list = br_i.down # perform analysis on this state
                stop_br = True
                break 

            elif br_i.up_state == 'surv' and br_i.down_state == 'fail' :

                comp_bnb, st_bnb_up = get_comp_st_for_next_bnb( up_dict, down_dict, rules, rules_st )
                brs_new_i = decomp_to_two_branches( br_i, comp_bnb, st_bnb_up, comps_name )
                
                cr_inds_up_new_i = []
                cr_inds_down_new_i = []
                for b in brs_new_i:
                    up_dict = comps_st_list_to_dict( b.up, comps_name )
                    cr_inds1, cst_state_up = get_compat_rules( up_dict, rules, rules_st )

                    if cst_state_up == 'unk' and len( cr_inds1 ) == 0:
                        cst_list = b.up # perform analysis on this state
                        stop_br = True
                        break 

                    else:
                        b.up_state = cst_state_up

                    
                    down_dict = comps_st_list_to_dict( b.down, comps_name )
                    cr_inds1, cst_state_down = get_compat_rules( down_dict, rules, rules_st )

                    if cst_state_down == 'unk' and len( cr_inds1 ) == 0:
                        cst_list = b.down # perform analysis on this state
                        stop_br = True
                        break 

                    else:
                        b.down_state = cst_state_down
                        if cst_state_down == cst_state_up:
                            b.is_complete = True

                        brs_new.append(b)
                
                if stop_br == True:
                    break

            elif br_i.up_state != 'unk' and br_i.up_state == br_i.down_state:
                brs_new.append(br_i)

            elif br_i.is_complete == True:
                brs_new.append(br_i)

            else:
                ddd = 1


        if stop_br == False:        
            brs = copy.deepcopy(brs_new)

        else:
            break


    cst_dict = comps_st_list_to_dict( cst_list, comps_name )

    no_sf += 1
    sys_val1, sys_st1, min_comps_st1 = sys_fun( cst_dict )
    sys_res = pd.concat( [sys_res,
                        pd.DataFrame( {'sys_val': [sys_val1], 'comps_st': [cst_dict], 'comps_st_min': [min_comps_st1]} )],
                        ignore_index = True )
    
    if sys_st1 == 'surv':
        if min_comps_st1 is not None:
            rs_new = min_comps_st1
        else:
            rs_new = { k:v for k,v in cst_dict.items() if v > 1 } # the rule is the same as up_dict_i but includes only components whose state is greater than the worst one (i.e. 1)
        
        rules, rules_st = add_a_new_rule( rules, rules_st, rs_new, 'surv' )

    else: # sys_st_i == 'fail'
        if min_comps_st1 is not None:
            rf_new = min_comps_st1
        else:
            rf_new = { k:v for k,v in cst_dict.items() if v < len(vari[k].B[0]) } # the rule is the same as up_dict_i but includes only components whose state is less than the best one

        rules, rules_st = add_a_new_rule( rules, rules_st, rf_new, 'fail' )
        
            

Iteration:  1
Known rules: 
Branching stage:  1
Branch #:  1
Branch:  Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=unk, up_state=unk, down_val=None, up_val=None
Iteration:  2
Known rules: 
surv {'e2': 3, 'e5': 3}
Branching stage:  1
Branch #:  1
Branch:  Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=unk, up_state=surv, down_val=None, up_val=None
Iteration:  3
Known rules: 
surv {'e2': 3, 'e5': 3}
fail {'e1': 1, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 1, 'e6': 1}
Branching stage:  1
Branch #:  1
Branch:  Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=fail, up_state=surv, down_val=None, up_val=None
Iteration:  4
Known rules: 
fail {'e1': 1, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 1, 'e6': 1}
surv {'e2': 2, 'e5': 3}
Branching stage:  1
Branch #:  1
Branch:  Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=fail, up_state=surv, down_val=None, up_val=None

### Results

In [38]:
print(no_iter)
print(no_sf)
print(len(rules))
print(len(brs))

print(rules)
print(rules_st)
print(brs)

print(sys_res)


26
26
10
16
[{'e1': 3, 'e3': 3, 'e5': 3}, {'e1': 2, 'e2': 1}, {'e2': 2, 'e6': 3, 'e4': 3}, {'e2': 1, 'e5': 2}, {'e2': 2, 'e5': 2}, {'e2': 3, 'e6': 2, 'e4': 3}, {'e2': 1, 'e3': 2}, {'e5': 1, 'e6': 1}, {'e2': 2, 'e5': 1, 'e6': 2}, {'e4': 2, 'e5': 1}]
['surv', 'fail', 'surv', 'fail', 'surv', 'surv', 'fail', 'fail', 'fail', 'fail']
[Branch(down=[1, 1, 1, 1, 1, 1], up=[2, 1, 3, 3, 3, 3], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None, Branch(down=[1, 2, 1, 1, 1, 1], up=[2, 3, 3, 3, 1, 1], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None, Branch(down=[1, 2, 1, 1, 1, 2], up=[2, 3, 3, 2, 1, 3], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None, Branch(down=[1, 2, 1, 3, 1, 2], up=[2, 2, 3, 3, 1, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None, Branch(down=[1, 2, 1, 3, 1, 3], up=[2, 2, 3, 3, 1, 3], is_complete=True, down_state=surv, up_state=surv, down_val=None, up_val=None, Bran